Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32) # -1:remaining dimension is inferred from array length
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32) # 1-hot encoding (examples x num_labels)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5 
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels)) # 4-D array - having multiple 3D input examples
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1)) # we have [depth] number of (5 x 5 x 1) kernels 
  layer1_biases = tf.Variable(tf.zeros([depth])) # static bias for each kernel, [depth] number in total 
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1)) # 3D kernel (5 x 5 x 16 weights), with [depth] number of them
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME') # stride array - stride for each input dim
    hidden = tf.nn.relu(conv + layer1_biases) # output is a ((batch x) 14 x 14 x [depth]) layer (same padding (2)), stride 2 
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases) # output is a ((batch x) 7 x 7 x [depth]) layer
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]]) # reshape to [batch, 7 x7 x [depth]]
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) # connect conv layer to 64-node hidden layer
    return tf.matmul(hidden, layer4_weights) + layer4_biases # fully connect hidden layer to output layer 
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 150

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.130406
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.704386
Minibatch accuracy: 37.5%
Validation accuracy: 59.7%
Minibatch loss at step 100: 1.005461
Minibatch accuracy: 68.8%
Validation accuracy: 73.9%
Test accuracy: 82.9%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
# set up the model; this is an answer to both problem 1 and 2.
# memory is limited; cannot run this on current computer

batch_size = 4
image_size = 28
num_channels = 1
patch_size = 5
depth = 16
num_hidden = 64
num_labels = 10
pool_size = 2
beta = 0.0005

graph = tf.Graph()
with graph.as_default():
    # Input Data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    # Variables
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1)) # filter1
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1)) # filter2
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size//4 * image_size//4 * depth, num_hidden], stddev=0.1)) # to hidden
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=np.sqrt(2.0/(num_hidden)))) # hidden to output
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    
    # Model
    def modelp1(data, dropout=False):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases) # produces batch_size x image_size x image_size x depth tensor
        pool = tf.nn.max_pool(hidden, [1, pool_size, pool_size, 1], [1, 2, 2, 1], padding='SAME')
        conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden, [1, pool_size, pool_size, 1], [1, 2, 2, 1], padding='SAME')
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]]) # reshape to batch x (h x w x d)
        if dropout: 
            # dropout on fully connected layer, training only
            hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases), 0.5) 
        else:
            hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) # to hidden layer
        connected = tf.matmul(hidden, layer4_weights) + layer4_biases # hidden to output
        return connected
    
    # Training
    logits = modelp1(tf_train_dataset, dropout=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * (
        tf.nn.l2_loss(layer3_weights) + 
        tf.nn.l2_loss(layer4_weights))
    
    # Optimizer
    initial_learning_rate = 0.05
    global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
    # decay every 5000 steps with a base of 0.96, staircase --> discrete decaying steps 
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, 5000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions
    tf_train_predictions = tf.nn.softmax(logits)
    tf_valid_predictions = tf.nn.softmax(modelp1(valid_dataset, dropout=False))
    tf_test_predictions = tf.nn.softmax(modelp1(test_dataset, dropout=False))

In [ ]:
# run the operation
import time
num_steps = 150

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('training started!\n')
    start = time.clock()
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {
            tf_train_dataset : batch_data,
            tf_train_labels : batch_labels
        }
        _, l, pred = session.run([optimizer, loss, tf_train_predictions], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Step: %d' % step)
            print('Time elapsed:', time.clock() - start, 's')
            print('Loss: %.2f' % l)
            print('Batch training accuracy: %.1f%%' % accuracy(pred, batch_labels))
            print('Batch validation accuracy: %.1f%%' % accuracy(tf_valid_predictions.eval(), valid_labels))
            print('------')
    print('Finished training\n------')
    print('Test accuracy: %.1f%%' % accuracy(tf_test_predictions.eval(), test_labels))
    print('Total training time:', time.clock() - start, 's')

training started!

Step: 0
Time elapsed: 0.03283 s
Loss: 6.86
Batch training accuracy: 25.0%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---